In [1]:
Pkg.add("RCall")

INFO: Cloning cache of AxisArrays from https://github.com/JuliaArrays/AxisArrays.jl.git
INFO: Cloning cache of CategoricalArrays from https://github.com/JuliaData/CategoricalArrays.jl.git
INFO: Cloning cache of Combinatorics from https://github.com/JuliaMath/Combinatorics.jl.git
INFO: Cloning cache of DataArrays from https://github.com/JuliaStats/DataArrays.jl.git
INFO: Cloning cache of DataFrames from https://github.com/JuliaStats/DataFrames.jl.git
INFO: Cloning cache of DataStructures from https://github.com/JuliaCollections/DataStructures.jl.git
INFO: Cloning cache of FileIO from https://github.com/JuliaIO/FileIO.jl.git
INFO: Cloning cache of GZip from https://github.com/JuliaIO/GZip.jl.git
INFO: Cloning cache of IntervalSets from https://github.com/JuliaMath/IntervalSets.jl.git
INFO: Cloning cache of IterTools from https://github.com/JuliaCollections/IterTools.jl.git
INFO: Cloning cache of NamedArrays from https://github.com/davidavdav/NamedArrays.jl.git
INFO: Cloning cache of Null

In [1]:
using RCall

INFO: Precompiling module RCall.
INFO: Using R installation at /Library/Frameworks/R.framework/Resources


In [2]:
R"library(raster)"

RCall.RObject{RCall.StrSxp}
[1] "raster"    "sp"        "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"     


In [3]:
R"library(maptools)"

RCall.RObject{RCall.StrSxp}
 [1] "maptools"  "raster"    "sp"        "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     


In [4]:
R"library(maps)"

RCall.RObject{RCall.StrSxp}
 [1] "maps"      "maptools"  "raster"    "sp"        "stats"     "graphics" 
 [7] "grDevices" "utils"     "datasets"  "methods"   "base"     


In [6]:
cd("/Users/kaminarikiminorushi/polybox/Shared/0626")

In [7]:
R"solar1<-readShapePoly('solar1.shp')"

RCall.RObject{RCall.S4Sxp}
class       : SpatialPolygonsDataFrame 
features    : 90508 
extent      : -125.1, -66.5, 24.2, 49.7  (xmin, xmax, ymin, ymax)
coord. ref. : NA 
variables   : 1
names       : DNIANN 
min values  :   2482 
max values  :   8314 


In [8]:
R"y<-raster(ncols=587,nrows=256,xmn=-125.1,xmx=-66.5,ymn=24.2,ymx=49.7)"

RCall.RObject{RCall.S4Sxp}
class       : RasterLayer 
dimensions  : 256, 587, 150272  (nrow, ncol, ncell)
resolution  : 0.09982964, 0.09960938  (x, y)
extent      : -125.1, -66.5, 24.2, 49.7  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0 



In [9]:
R"solargrid1<-rasterize(solar1, y, 'DNIANN')"

RCall.RObject{RCall.S4Sxp}
class       : RasterLayer 
dimensions  : 256, 587, 150272  (nrow, ncol, ncell)
resolution  : 0.09982964, 0.09960938  (x, y)
extent      : -125.1, -66.5, 24.2, 49.7  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0 
data source : in memory
names       : layer 
values      : 2482, 8314  (min, max)



In [10]:
R"solargrid1[is.na(solargrid1)==TRUE]<-0"


RCall.RObject{RCall.RealSxp}
[1] NaN


In [11]:
R"solarpoint<-rasterToPoints(solargrid1, fun=NULL, spatial=FALSE)";
R"x<-solarpoint[,1]";
R"y<-solarpoint[,2]";
R"xy<-cbind(x,y)";
R"solar_value<-solarpoint[,3]";


In [15]:
R"CountyBoundary<-readShapePoly('US_county_2000-simple-latlon.shp');"


RCall.RObject{RCall.S4Sxp}
class       : SpatialPolygonsDataFrame 
features    : 3141 
extent      : -179.1473, 179.7785, 18.91393, 71.38961  (xmin, xmax, ymin, ymax)
coord. ref. : NA 
variables   : 20
names       : DECADE,  NHGISNAM, NHGISST, NHGISCTY, ICPSRST, ICPSRCTY,  ICPSRNAM, STATENAM, ICPSRSTI, ICPSRCTYI, ICPSRFIP, STATE, COUNTY,  PID,    X_CENTROID, ... 
min values  :   2000, Abbeville,     010,     0010,       1,       10, ABBEVILLE,  Alabama,        0,         0,        0,   010,   0010,    0,  0.000000e+00, ... 
max values  :   2000,   Ziebach,     560,     8400,      82,      990,   ZIEBACH,  Wyoming,       82,      8400,        0,   560,   8400, 3109, -9.999160e+05, ... 


In [16]:
R"county <-matrix(numeric(0), 3109,0);"
R"for(i in 1:3109){
  countyboundary <- subset(CountyBoundary,PID==i)
  temp_JJJ <- raster(res=10) #actually I don't the influence of setting resolution
  extent(temp_JJJ) <- extent(countyboundary)
  out <- rasterize(countyboundary, temp_JJJ,'PID')
  countyresult<- rasterize(xy,out, solar_value)
  county[i]<-cellStats(countyresult,mean)
}"

RCall.RObject{RCall.NilSxp}
NULL


In [18]:
R"county"

RCall.RObject{RCall.RealSxp}
   [1] 5781.450 4591.375 4595.667 4393.750 3930.333 3873.917 4108.556 3700.500
   [9] 3688.400 3776.917 3778.067 3617.667 3861.250 3746.833 3549.000 3820.467
  [17] 3956.875 4029.375 3723.667 4118.667 3802.750 3683.167 3837.800 4003.720
  [25] 3743.667 3659.050 3832.333 4032.500 3961.800 3899.500 3649.375 3782.250
  [33] 3758.800 3415.100 3885.111 4145.929 3779.500 3808.250 3793.667 4086.800
  [41] 3940.444 3663.700 3957.583 3576.417 3805.733 3893.667 3984.524 3841.733
  [49] 4105.167 3995.708 3687.833 3810.111 3457.750 3785.917 3428.800 3883.167
  [57] 3710.750 3522.625 3625.444 3634.300 3427.944 3718.208 3849.812 4046.100
  [65] 4028.667 4085.267 3759.700 3746.800 3506.062 3824.733 4037.167 4192.120
  [73] 4161.093 4201.900 4229.533 4451.486 4193.875 4134.250 4132.222 4173.625
  [81] 4223.597 4417.964 4267.743 4184.714 4275.960 4364.914 4232.536 4126.886
  [89] 4147.278 4190.036 4200.917 4301.188 4198.292 4216.567 3835.333 3781.492
  [97] 3826.800 3709.52